# Meta-Ensemble: MTLR Ensemble + XGBoost AFT Ensemble

This notebook combines two survival models:
- an **MTLR ensemble**
- an **XGBoost AFT ensemble**

Both ensembles are assumed to produce risk scores on the **same validation or test set**.

We build a **meta-ensemble** by averaging the ranks of their risk scores (rank-based ensembling).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sksurv.util import Surv
from sksurv.metrics import concordance_index_ipcw

plt.style.use('default')

# === CONFIGURATION ===
# Paths to the prediction files produced by the MTLR and XGB AFT ensembles.
# Each file must contain at least: 'ID' and 'risk_score'.

MTLR_PRED_PATH = "submission_mtlr_ensemble_nestedcv.csv"  # adapt if needed
XGB_PRED_PATH = "../submissions/submission_xgb_aft_ensemble_nestedcv.csv"  # adapt if needed

# Optional: path to a labeled validation set to evaluate IPCW C-index
# The file should contain columns: 'ID', 'OS_STATUS', 'OS_YEARS'.

VAL_DATA_PATH = "../../data/val_enhanced.csv"  # adapt or set to None if not available
TIME_COL = "OS_YEARS"
EVENT_COL = "OS_STATUS"
TAU_CINDEX = 7.0


In [ ]:
# === LOAD PREDICTIONS ===
mtlr_pred = pd.read_csv(MTLR_PRED_PATH)
xgb_pred = pd.read_csv(XGB_PRED_PATH)

print("MTLR predictions:")
print(mtlr_pred.head())
print("\nXGB AFT predictions:")
print(xgb_pred.head())

# Sanity check on columns
assert "ID" in mtlr_pred.columns and "risk_score" in mtlr_pred.columns, "MTLR file must contain 'ID' and 'risk_score' columns."
assert "ID" in xgb_pred.columns and "risk_score" in xgb_pred.columns, "XGB file must contain 'ID' and 'risk_score' columns."


In [ ]:
# === MERGE PREDICTIONS ON ID ===
merged = (
    mtlr_pred.rename(columns={"risk_score": "risk_mtlr"})
    .merge(xgb_pred.rename(columns={"risk_score": "risk_xgb"}), on="ID", how="inner")
)

print(f"Merged shape: {merged.shape}")
print(merged.head())


In [ ]:
# === RANK-BASED AVERAGE ENSEMBLING ===

def rank_based_ensemble(df, risk_cols):
    """Compute rank-based ensemble from given risk columns.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing risk score columns.
    risk_cols : list of str
        Names of the risk score columns to ensemble.

    Returns
    -------
    np.ndarray
        Average ranks (1 = lowest risk, N = highest risk).
    """
    n = len(df)
    ranks = []
    for col in risk_cols:
        # ascending: lowest risk -> rank 1
        order = np.argsort(df[col].values)
        rank = np.empty_like(order, dtype=float)
        rank[order] = np.arange(1, n + 1)
        ranks.append(rank)

    ranks = np.vstack(ranks)  # shape (n_models, n_samples)
    return ranks.mean(axis=0)

risk_cols = ["risk_mtlr", "risk_xgb"]
merged["risk_ensemble_rank"] = rank_based_ensemble(merged, risk_cols)

# Optional: normalize ensemble risk between 0 and 1 for interpretability
r_min = merged["risk_ensemble_rank"].min()
r_max = merged["risk_ensemble_rank"].max()
merged["risk_ensemble_rank_norm"] = (
    (merged["risk_ensemble_rank"] - r_min) / (r_max - r_min + 1e-8)
)

print("\nMerged with ensemble risk:")
print(merged.head())


In [ ]:
# === OPTIONAL: EVALUATE IPCW C-INDEX ON A LABELED VALIDATION SET ===

def compute_ipcw_cindex(time_train, event_train, time_test, event_test, risk_scores, tau):
    y_train_struct = Surv.from_arrays(event=event_train.astype(bool), time=time_train)
    y_test_struct = Surv.from_arrays(event=event_test.astype(bool), time=time_test)
    return concordance_index_ipcw(y_train_struct, y_test_struct, risk_scores, tau=tau)[0]

try:
    val_df = pd.read_csv(VAL_DATA_PATH)
    print(f"\nLoaded validation data: {val_df.shape}")

    assert TIME_COL in val_df.columns and EVENT_COL in val_df.columns and "ID" in val_df.columns,
        "Validation data must contain 'ID', time and event columns."

    # Align merged predictions with validation labels
    val_merged = val_df[["ID", TIME_COL, EVENT_COL]].merge(merged, on="ID", how="inner")
    print(f"Aligned validation + predictions shape: {val_merged.shape}")

    time_all = val_merged[TIME_COL].to_numpy(dtype=float)
    event_all = val_merged[EVENT_COL].to_numpy(dtype=bool)

    # For IPCW, we need a "training" sample to estimate weights.
    # Here, we simply reuse the same validation data as reference.
    time_train_ref = time_all
    event_train_ref = event_all

    # MTLR C-index
    cindex_mtlr = compute_ipcw_cindex(
        time_train_ref,
        event_train_ref,
        time_all,
        event_all,
        val_merged["risk_mtlr"].values,
        tau=TAU_CINDEX,
    )

    # XGB C-index
    cindex_xgb = compute_ipcw_cindex(
        time_train_ref,
        event_train_ref,
        time_all,
        event_all,
        val_merged["risk_xgb"].values,
        tau=TAU_CINDEX,
    )

    # Ensemble C-index
    cindex_ens = compute_ipcw_cindex(
        time_train_ref,
        event_train_ref,
        time_all,
        event_all,
        val_merged["risk_ensemble_rank"].values,
        tau=TAU_CINDEX,
    )

    print("\nIPCW C-index on validation:")
    print(f"  MTLR ensemble      : {cindex_mtlr:.4f}")
    print(f"  XGB AFT ensemble   : {cindex_xgb:.4f}")
    print(f"  Rank-avg ensemble  : {cindex_ens:.4f}")

except FileNotFoundError:
    print("Validation data file not found. Skipping IPCW C-index evaluation.")
except AssertionError as e:
    print(f"Validation data format issue: {e}")
except Exception as e:
    print(f"Unexpected error during evaluation: {e}")


In [ ]:
# === SAVE FINAL META-ENSEMBLE PREDICTIONS ===

final_submission = merged[["ID", "risk_ensemble_rank_norm"]].rename(
    columns={"risk_ensemble_rank_norm": "risk_score"}
)

output_path = "../submissions/submission_meta_ensemble_rank_avg.csv"
final_submission.to_csv(output_path, index=False)
print(f"Meta-ensemble predictions saved to: {output_path}")
print(final_submission.head())
